In [202]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import minimize, least_squares
from sklearn.linear_model import LinearRegression, Lasso
from python_module.pricing_model import SABRModel, BlackScholesModel

In [236]:
# -------
# PARAMETERS
# -------
F = 100
T = 250/250
alpha = 0.1
beta = 1
rho = -0.
nu = 0.00001
n_steps = 250
n_paths = 100

In [237]:
# -------
# MONTECARLO
# -------
spot, vol = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths, seed=False)
spot = np.round(spot, 2)

In [238]:
x = (spot[0].pct_change().pow(2)) / (1/250)

In [239]:
np.sqrt(x.mean())

0.09771500689657704

In [222]:
df = vol[0].to_frame('alpha')
log_returns = np.log(spot[0]/spot[0].shift())
df['ewm'] = np.sqrt(log_returns.pow(2).ewm(span=40).mean()*250)

In [223]:
from pykalman import KalmanFilter
kf = KalmanFilter(transition_matrices = [1],
              observation_matrices = [1],
              initial_state_mean = 0,
              initial_state_covariance = 1,
              observation_covariance=1,
              transition_covariance=.0001)

mean, cov = kf.filter(log_returns[1:].pow(2).values)
mean, std = mean.squeeze(), np.std(cov.squeeze())
df['kalman'] = pd.Series(index=log_returns.index[1:], data=np.sqrt(mean*250))

In [224]:
px.line(df)

In [219]:
df.iloc[-60:].diff().corr()

,alpha,ewm,kalman
alpha,1.000000,-0.145638,-0.221056
ewm,-0.145638,1.000000,0.963792
kalman,-0.221056,0.963792,1.000000
